In [1]:
from google.colab import files
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
import string
import gensim
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-11-05 11:16:29--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-05 11:16:30--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-05 11:16:30--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [3]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [4]:
folklore = pd.read_csv('08-folklore_deluxe_version.csv')
evermore = pd.read_csv('09-evermore_deluxe_version.csv')
lover = pd.read_csv('07-lover.csv')
reputation = pd.read_csv('06-reputation.csv')
the1989 = pd.read_csv('05-1989_deluxe.csv')
red = pd.read_csv('04-red_deluxe_edition.csv')
speak = pd.read_csv('03-speak_now_deluxe_package.csv')
fearless = pd.read_csv('02-fearless_taylors_version.csv')
taylor = pd.read_csv('01-taylor_swift.csv')
midnights = pd.read_csv('10_midnights.csv')

In [5]:
df = pd.concat([folklore, evermore, lover,reputation,the1989,red,speak,fearless,taylor], axis=0)
df.head()

,album_name,track_title,track_n,lyric,line
0,folklore (deluxe version),the 1,1,"I'm doing good, I'm on some new shit",1
1,folklore (deluxe version),the 1,1,"Been saying ""yes"" instead of ""no""",2
2,folklore (deluxe version),the 1,1,"I thought I saw you at the bus stop, I didn't ...",3
3,folklore (deluxe version),the 1,1,I hit the ground running each night,4
4,folklore (deluxe version),the 1,1,I hit the Sunday matinée,5


In [6]:
df = df[['track_title', 'lyric']]

In [7]:
df = pd.concat([df, midnights], axis = 0)

In [8]:
df.tail()

,track_title,lyric
15,Paris,Your ex-friend's sister met someone at a club ...
16,High Infidelity,Lock broken\nSlur spoken\nWound open\nGame tok...
17,Glitch,We were supposed to be just friends\nYou don't...
18,"Would've, Could've, Should've","If you would've blinked, then I would've\nLook..."
19,Dear Reader,Dear reader\nIf it feels like a trap\nYou're a...


In [ ]:
# preprocessing - zmienić didn't na did not itd

In [9]:
import re

In [10]:
df['lyric'] = df['lyric'].apply(lambda x: re.sub(r"\d+", "", x))

In [11]:
df['lyric'] = df['lyric'].apply(lambda x: re.sub(r"[.]?-[.]?", " ", x))

In [12]:
df['lyric'] = df['lyric'].apply(lambda x: x.replace("\n", " "))

In [13]:
translate_table = dict((ord(char), None) for char in string.punctuation) 
df['lyric'] = df['lyric'].apply(lambda x: x.translate(translate_table))

In [14]:
df.tail()

,track_title,lyric
15,Paris,Your ex friends sister met someone at a club a...
16,High Infidelity,Lock broken Slur spoken Wound open Game token ...
17,Glitch,We were supposed to be just friends You dont l...
18,"Would've, Could've, Should've",If you wouldve blinked then I wouldve Looked a...
19,Dear Reader,Dear reader If it feels like a trap Youre alre...


In [15]:
df['lyric'] = df['lyric'].apply(lambda x: x.lower())

In [16]:
def get_pos(word):

  #Słownik mapujący symbole części mowy zwrócone przez tagger do ich ekwiwalentów akceptowalnych przez lemmatizer
    tag_map = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

  #Tagowanie części mowy i pobieranie jej symbolu
    tag = nltk.pos_tag([word])[0][1][0].upper()

  #Zwracanie zmapowanej wartości - w przypadku braku wartością domyślną jest rzeczownik
    return tag_map.get(tag, wordnet.NOUN)

In [17]:
lemmatizer = WordNetLemmatizer()

In [18]:
def lemmatize(text):
    return ' '.join(lemmatizer.lemmatize(word, get_pos(word)) for word in nltk.word_tokenize(text))

In [19]:
df['lyric'] = df['lyric'].apply(lambda x: x.replace("\u2005", " "))

In [20]:
df['lyric'] = df['lyric'].apply(lambda x: lemmatize(x))

In [21]:
all_lyrics = ' '.join(list(df['lyric']))

In [22]:
words = nltk.word_tokenize(all_lyrics)

words=[word.lower() for word in words]
vocab = set(words)

In [23]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(vocab)

In [24]:
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
    vocab_size = len(word_index) + 1
      
    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))
  
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
  
    return embedding_matrix_vocab

In [25]:
# matrix for vocab: word_index
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab(
    './glove.6B.50d.txt', tokenizer.word_index,
  embedding_dim)
  
print("Dense vector for first word is => ",
      embedding_matrix_vocab[1])

Dense vector for first word is =>  [ 0.39660001  0.59820998  0.11068    -0.45436001  1.27590001  1.60520005
 -1.06620002  0.1954     -0.60737997  0.0081428  -0.07934     0.37204
 -0.25450999 -0.45528001  1.23819995  0.054297   -0.66169     0.0635
  0.30669001  0.37075001  0.12926     1.77509999 -0.06796    -0.26431
  0.85355997 -1.91760004  0.074934   -0.32350001  0.38055     0.02172
  1.6803      0.14786001 -0.15987    -0.087453    0.25015     0.030113
 -0.12493     0.30737999  0.69665998 -0.40256    -0.16874     0.88116997
 -0.15933999 -0.33761999 -0.13911    -0.97544998 -0.88182002 -1.45749998
  0.29831001  0.23416001]


In [26]:
nn = NearestNeighbors(n_neighbors=4, algorithm='ball_tree', metric='euclidean')

In [27]:
nn.fit(embedding_matrix_vocab)

NearestNeighbors(algorithm='ball_tree', metric='euclidean', n_neighbors=4)

In [28]:
results = nn.kneighbors(embedding_matrix_vocab[[3]])

In [29]:
most_suitable = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(results[1][0][0])]

In [30]:
most_suitable

'screw'

In [31]:
vectorizer = TfidfVectorizer(
    analyzer='word', 
    stop_words='english', 
    ngram_range = (1,1), 
    #max_df=0.5, 
    use_idf=True, 
    smooth_idf=True
    #max_features=1000
    )

In [ ]:
df.head()

,album_name,track_title,track_n,lyric,line
0,folklore (deluxe version),the 1,1,Im do good Im on some new shit,1
1,folklore (deluxe version),the 1,1,Been say yes instead of no,2
2,folklore (deluxe version),the 1,1,I thought I saw you at the bus stop I didnt th...,3
3,folklore (deluxe version),the 1,1,I hit the ground run each night,4
4,folklore (deluxe version),the 1,1,I hit the Sunday matinée,5


In [32]:
grouped = df[['track_title', 'lyric']].groupby('track_title').agg(list).reset_index()

In [33]:
grouped['lyric'] = grouped['lyric'].apply(lambda x: ' '.join(x))

In [34]:
grouped.tail()

,track_title,lyric
178,​r​ight where you left me,friend break up friend get married stranger ge...
179,​the lakes,be it romantic how all my elegy eulogize me im...
180,​the last great american dynasty,rebekah rode up on the afternoon train it be s...
181,​tolerate it,i sit and watch you reading with your head low...
182,​’tis the damn season,if i want to know who you be hang with while i...


In [35]:
tfidf_matrix  = vectorizer.fit_transform(grouped['lyric'].tolist() )

In [36]:
tfidf_features = sorted(vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [37]:
df_tfidf = pd.DataFrame(data = tfidf_matrix.toarray(),index = range(len(grouped)),columns = tfidf_features)
df_tfidf

,abigail,absent,absurd,abusive,accent,accident,accidental,account,accuse,ace,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.082846,0.037468,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.046119,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.043059,0.020324,0.0,0.0
179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.056988,0.0,0.0


In [38]:
result = pd.concat([grouped, df_tfidf], axis=1, join="inner")

In [39]:
result.head()

,track_title,lyric,abigail,absent,absurd,abusive,accent,accident,accidental,account,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
0,...Ready for It?,knew he be a killer first time that i saw him ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.082846,0.037468,0.000000,0.0,0.0
1,22,it feel like a perfect night to dress up like ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,A Perfectly Good Heart,why would you wan na break a perfectly good he...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,A Place In This World,i dont know what i want so dont ask me cause i...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,Afterglow,i blew thing out of proportion now youre blue ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.046119,0.0,0.0


In [40]:
result.nlargest(3,'snow')

,track_title,lyric,abigail,absent,absurd,abusive,accent,accident,accidental,account,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
98,Snow On The Beach,one night a few moon ago i saw fleck of what c...,0.0,0.0,0.0,0.045659,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000
61,It’s Nice to Have a Friend,ooh ooh school bell ring walk me home sidewalk...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.05564
117,The Best Day (Taylor’s Version),im five year old it get cold ive get my big co...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.102132,0.0,0.00000


In [44]:
metadata = pd.read_csv('spotify_taylorswift.csv')

In [45]:
metadata = metadata[['name', 'danceability', 'acousticness', 'energy', 'liveness', 'valence']]

In [41]:
metadata_mid = pd.read_csv('midnights-spotify.csv')

In [46]:
final_metadata = pd.concat([metadata, metadata_mid], axis=0)

In [48]:
final_metadata.head()

,name,danceability,acousticness,energy,liveness,valence
0,Tim McGraw,0.580,0.575,0.491,0.1210,0.425
1,Picture To Burn,0.658,0.173,0.877,0.0962,0.821
2,Teardrops On My Guitar - Radio Single Remix,0.621,0.288,0.417,0.1190,0.289
3,A Place in this World,0.576,0.051,0.777,0.3200,0.428
4,Cold As You,0.418,0.217,0.482,0.1230,0.261


In [49]:
final_metadata['dance'] = 0.5 * final_metadata['danceability'] + 0.25 * final_metadata['energy'] + 0.25 * final_metadata['liveness']

In [52]:
final_metadata = final_metadata[['name','dance', 'acousticness', 'valence']]

In [53]:
final = pd.merge(
    final_metadata,
    result,
    how="inner",
    on=None,
    left_on="name",
    right_on="track_title",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [56]:
final.head()

,name,dance_x,acousticness,valence,track_title,lyric,abigail,absent,absurd,abusive,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
0,22,0.524675,0.00215,0.668,22,it feel like a perfect night to dress up like ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,A Perfectly Good Heart,0.461250,0.00349,0.268,A Perfectly Good Heart,why would you wan na break a perfectly good he...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,Afterglow,0.518750,0.13000,0.399,Afterglow,i blew thing out of proportion now youre blue ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046119,0.0,0.000000
3,All Too Well,0.485250,0.03820,0.337,All Too Well,i walk through the door with you the air be co...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.099881
4,Anti-Hero,0.515500,0.13300,0.519,Anti-Hero,i have this thing where i get old but just nev...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039772,0.0,0.000000


In [57]:
features = ['dance_x', 'acousticness', 'valence']

In [561]:
example = {'homesick': 1}

In [562]:
# matrix for vocab: word_index
embedding_dim = 50
embedding = embedding_for_vocab(
    './glove.6B.50d.txt', example,
  embedding_dim)

In [563]:
resultt = nn.kneighbors(embedding[[1]])

In [564]:
all_results = list(resultt[1][0])

In [565]:
top_words = [list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(x)] for x in all_results]

In [566]:
new_features = features + top_words

In [567]:
new_features

['dance_x',
 'acousticness',
 'valence',
 'lonely',
 'dizzy',
 'unmoored',
 'hopeless']

In [568]:
#exception handling jesli czegos nie ma w indexie to probuje znalezc kolejne slowo i nim zastapic
# ew jesli to tylko to everyone to wywalic
with_words = final[new_features]

In [569]:
with_words['word_mean'] = 0.6 * with_words.iloc[:, 3] + 0.2 * with_words.iloc[:, 4] + 0.15 * with_words.iloc[:, 5] + 0.05 * with_words.iloc[:, 6]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [570]:
with_words = with_words[['dance_x', 'acousticness', 'valence', 'word_mean']]

In [571]:
with_words

,dance_x,acousticness,valence,word_mean
0,0.524675,0.00215,0.668,0.059079
1,0.461250,0.00349,0.268,0.000000
2,0.518750,0.13000,0.399,0.000000
3,0.485250,0.03820,0.337,0.000000
4,0.515500,0.13300,0.519,0.000000
...,...,...,...,...
113,0.311225,0.81400,0.112,0.000000
114,0.447050,0.88800,0.374,0.000000
115,0.507250,0.75000,0.163,0.000000
116,0.411750,0.31200,0.408,0.000000


In [572]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
for feature in list(with_words.columns):
    with_words[[feature]] = scaler.fit_transform(with_words[[feature]])

In [573]:
nn_final = NearestNeighbors(n_neighbors=20, algorithm='ball_tree')

In [574]:
nn_final.fit(with_words.values)

NearestNeighbors(algorithm='ball_tree', n_neighbors=20)

In [575]:
result_final = nn_final.kneighbors([[0.3,0.5,0.1,1]])

In [576]:
all_results = list(result_final[1][0])

In [577]:
songs = [final['name'][i] for i in all_results]

In [578]:
songs

['Sad Beautiful Tragic',
 'The Outside',
 'Enchanted',
 'Dear Reader',
 'This Love',
 'Last Kiss',
 'Mastermind',
 'Ours',
 'Invisible',
 'White Horse (Taylor’s Version)',
 'Cold As You',
 'august',
 'Snow On The Beach',
 'Midnight Rain',
 'my tears ricochet',
 'Lover',
 'Dear John',
 'Lavender Haze',
 'this is me trying',
 'Bigger Than The Whole Sky']

In [579]:
nn_word = NearestNeighbors(n_neighbors=30, algorithm='ball_tree')

In [580]:
nn_word.fit(with_words[['word_mean']].values)

NearestNeighbors(algorithm='ball_tree', n_neighbors=30)

In [581]:
rs = nn_word.kneighbors([[1]])

In [582]:
all_results = list(rs[1][0])
songs_word = [final['name'][i] for i in all_results]
songs_word

['The Outside',
 '22',
 'Sad Beautiful Tragic',
 'High Infidelity',
 'Enchanted',
 'Lavender Haze',
 'Miss Americana & The Heartbreak Prince',
 'Shake It Off',
 'This Love',
 'Picture To Burn',
 'Sparks Fly',
 'Starlight',
 'Superstar (Taylor’s Version)',
 'The Archer',
 'The Lucky One',
 'The Moment I Knew',
 'The Man',
 'Stay Beautiful',
 'The Best Day (Taylor’s Version)',
 'The Way I Loved You (Taylor’s Version)',
 'Snow On The Beach',
 'So It Goes...',
 'Speak Now',
 "Should've Said No",
 'Style',
 'Superman',
 'Sweet Nothing',
 'The Great War',
 'The Last Time',
 'Tell Me Why (Taylor’s Version)']

In [583]:
def common_elements(list1, list2):
    return [element for element in list1 if element in list2]

In [584]:
songs_fin = common_elements(songs, songs_word)

In [585]:
songs_fin

['Sad Beautiful Tragic',
 'The Outside',
 'Enchanted',
 'This Love',
 'Snow On The Beach',
 'Lavender Haze']

In [586]:
songs_no_words = [x for x in songs if x not in songs_fin]

In [587]:
if len(songs_fin) == 10:
  songs_10 = songs_fin
elif len(songs_fin) > 10:
  songs_10 = songs_fin[:10]
else:
  songs_10 = songs_fin + songs_no_words[:10-len(songs_fin)]

In [588]:
songs_10

['Sad Beautiful Tragic',
 'The Outside',
 'Enchanted',
 'This Love',
 'Snow On The Beach',
 'Lavender Haze',
 'Dear Reader',
 'Last Kiss',
 'Mastermind',
 'Ours']